### RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

c:\Users\Roshan\Desktop\RAG\RAG-Tutorials\rag_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Read all the pdf's inside the directory

In [5]:
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
                doc.metadata['time_loading']=datetime.now().strftime('%Y-%d-%m')
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 5 PDF files to process

Processing: paint  (1).pdf
  ✓ Loaded 2 pages

Processing: paint  (2).pdf
  ✓ Loaded 758 pages

Processing: paint  (3).pdf
  ✓ Loaded 2 pages

Processing: paint  (6).pdf
  ✓ Loaded 2 pages

Processing: Roshan_Chowrasia_Data_Scientist_Resume.pdf
  ✓ Loaded 2 pages

Total documents loaded: 766


In [10]:
all_pdf_documents[0].metadata

{'producer': 'pdfTeX-1.40.26',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2025-08-28T17:00:39+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2025-08-28T17:00:39+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0',
 'subject': '',
 'title': '',
 'trapped': '/False',
 'source': '..\\data\\pdf\\paint  (1).pdf',
 'total_pages': 2,
 'page': 0,
 'page_label': '1',
 'source_file': 'paint  (1).pdf',
 'file_type': 'pdf',
 'time_loading': '2025-28-10'}

### Text splitting get into chunks

In [11]:
def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [13]:
chunks=split_documents(all_pdf_documents)

Split 766 documents into 2338 chunks

Example chunk:
Content: ROSHAN CHOWRASIA ♂phone+91-8334913863
M.Tech. /envel⌢peroshanchowrasia12@gmail.com
Computational and Data Sciences (CDS) Indian Institute Of Science, Bangalore
/linkedinLinkedIn Profile
Summary
Skille...
Metadata: {'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-08-28T17:00:39+00:00', 'author': '', 'keywords': '', 'moddate': '2025-08-28T17:00:39+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdf\\paint  (1).pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'paint  (1).pdf', 'file_type': 'pdf', 'time_loading': '2025-28-10'}


In [14]:
len(chunks),len(chunks[0].page_content)

(2338, 980)

### embedding And vectorStoreDB

In [15]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### VectorStore

In [32]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            print(self.collection)
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 11690
Collection(name=pdf_documents)


In [18]:
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-08-28T17:00:39+00:00', 'author': '', 'keywords': '', 'moddate': '2025-08-28T17:00:39+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': '..\\data\\pdf\\paint  (1).pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'paint  (1).pdf', 'file_type': 'pdf', 'time_loading': '2025-28-10'}, page_content='ROSHAN CHOWRASIA ♂phone+91-8334913863\nM.Tech. /envel⌢peroshanchowrasia12@gmail.com\nComputational and Data Sciences (CDS) Indian Institute Of Science, Bangalore\n/linkedinLinkedIn Profile\nSummary\nSkilled data scientist with a solid foundation in machine learning, deep learning, and natural language\nprocessing. Proficient in Python, SQL, and frameworks like Scikit-learn and PyTorch. Experienced in\ndeveloping predictive models, performing statisti

### Convert the text to embeddings

In [20]:

texts=[doc.page_content for doc in chunks]
print(len(texts))
## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

2338
Generating embeddings for 2338 texts...


Batches: 100%|██████████| 74/74 [04:43<00:00,  3.83s/it]

Generated embeddings with shape: (2338, 384)


In [22]:
embeddings.shape[0]

2338

### store int he vector database

In [31]:
vectorstore.add_documents(chunks,embeddings)

Adding 2338 documents to vector store...
Successfully added 2338 documents to vector store
Total documents in collection: 11690


In [34]:
embeddings[0].shape

(384,)

In [26]:
texts_len=[len(s) for s in texts]
max(texts_len)

1000

### Retriever Pipeline From VectorStore --> Context

In [52]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager : EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if distance >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)



In [53]:
rag_retriever

In [54]:
# rag_retriever.retrieve("What is attention is all you need")
rag_retriever.retrieve("Tell me about Roshan Chowrasia IISc Bangalore Computational Data Science")

Retrieving documents for query: 'Tell me about Roshan Chowrasia IISc Bangalore Computational Data Science'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 32.17it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_2c900366_2317',
  'content': 'ROSHAN CHOWRASIA ♂phone+91-8334913863\nM.Tech. /envel⌢peroshanchowrasia12@gmail.com\nComputational Data Science (CDS) Indian Institute Of Science, Bangalore\n/linkedinLinkedIn Profile\nSummary\nSkilled data scientist with a solid foundation in machine learning, deep learning, and natural language\nprocessing. Proficient in Python, SQL, and frameworks like Scikit-learn and PyTorch. Experienced in\ndeveloping predictive models, performing statistical analysis, and building end-to-end data pipelines. Adept\nat deploying data-driven solutions and creating interactive dashboards using Power BI. Strong academic\nbackground with hands-on experience in projects involving NLP, time series forecasting, and classification\ntasks.\nEducation\n•Indian Institute Of Science, Bangalore 2022-2024\nM.Tech in Computational Data Science CGPA: 8.6\n•St Xavier’s College (Autonomous), Kolkata 2020-2022\nM.Sc. in Computer Science Percentage: 77.62\n•St Xavier’s Colle

### RAG Pipeline- VectorDB To LLM Output Generation

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

print(os.getenv("GROQ_API_KEY"))

In [17]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage

In [18]:
class GroqLLM:
    def __init__(self, model_name: str = "gemma2-9b-it", api_key: str =None):
        """
        Initialize Groq LLM
        
        Args:
            model_name: Groq model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
            api_key: Groq API key (or set GROQ_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        
        if not self.api_key:
            raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter.")
        
        self.llm = ChatGroq(
            groq_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024
        )
        
        print(f"Initialized Groq LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context
        
        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length
            
        Returns:
            Generated response string
        """
        
        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )
        
        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)
        
        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content
            
        except Exception as e:
            return f"Error generating response: {str(e)}"
        
    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting
        
        Args:
            query: User question
            context: Retrieved context
            
        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""
        
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"
    


In [ ]:
# Initialize Groq LLM (you'll need to set GROQ_API_KEY environment variable)
try:
    groq_llm = GroqLLM(api_key=os.getenv("GROQ_API_KEY"))
    print("Groq LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your GROQ_API_KEY environment variable to use the LLM.")
    groq_llm = None

Initialized Groq LLM with model: gemma2-9b-it
Groq LLM initialized successfully!


In [21]:
### get the context from the retriever and pass it to the LLM

rag_retriever.retrieve("Unified Multi-task Learning Framework")

Retrieving documents for query: 'Unified Multi-task Learning Framework'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 88.98it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_bd5cc745_61',
  'content': 'erage scores on CMTEB[ 22] and MTEB[ 23] benchmarks, ranking ﬁrst overall on both\nCMTEB and MTEB leaderboards, demonstrating the eﬀectiveness o f our approach.\nThe contributions of our work are summarized as follows:\n• We propose a uniﬁed multi-task learning framework that systematic ally coordi-\nnates both data processing and training pipelines, enhancing divers ity in datasets\nand eﬃciency in model training ;\n• We develop advanced data synthesis techniques powered by LLM, in cluding Para-\nphrasing, Data augmentation, and Hard negative generation. The se methods\nsigniﬁcantly enhance the quality of training corpora, thereby impro ving model’s\nrobustness and generalization capabilities;\n• We emply a two-stage training paradigm: Stage 1 focuses exclusively on retrieval\ncapability building, establishing strong foundational retrieval perf ormance; and\nstage 2 implements balanced training with controled retrieval/non-r etrieval task',
  '

### Integration Vectordb Context pipeline With LLM output

In [67]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant",temperature=0.0,max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [68]:
answer=rag_simple("What is attention mechanism?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is attention mechanism?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 40.02it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


The attention mechanism is a technique used in deep learning models to focus on specific parts of the input data that are relevant to the task at hand. It allows the model to selectively weigh the importance of different input elements, rather than treating all elements equally. This is particularly useful in tasks with long-term dependencies, such as the credit assignment problem mentioned in the context, where the model needs to attribute rewards to specific actions or moves.


### Enhanced RAG Pipeline Features

In [69]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("Hard Negative Mining Technqiues", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'Hard Negative Mining Technqiues'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 41.67it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


Answer: Hard Negative Mining Techniques are used to handle the issue of outliers or misclassified data points in classification problems, particularly when using the exponential error function. These techniques aim to reduce the impact of large negative values of ty(x) on the model's performance.
Sources: [{'source': 'paint  (2).pdf', 'page': 681, 'score': -0.3621065616607666, 'preview': 'negative values of ty(x) much more strongly than cross-entropy. In particular, we\nsee that for large negative values of ty, the cross-entropy grows linearly with |ty|,\nwhereas the exponential error function grows exponentially with |ty|. Thus the ex-\nponential error function will be much less robust...'}, {'source': 'paint  (2).pdf', 'page': 681, 'score': -0.3621065616607666, 'preview': 'negative values of ty(x) much more strongly than cross-entropy. In particular, we\nsee that for large negative values of ty, the cross-entropy grows linearly with |ty|,\nwhereas the exponential error function grows

In [72]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("Tell me about roshan chowrasia", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'Tell me about roshan chowrasia'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 32.26it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
ROSHAN CHOWRASIA
+91-8334913863
roshanchowrasia12@gmail.com
linkedin.com/in/roshan-chowrasia-625b3625b
Summary
Data Scientist with 1.5+ years of industrial experience specializing in machine learning, statistical
optimization, and time series forecast

ing. Proven expertise in predictive modeling, ETL automation, and data
pipeline development using Python, SQL, Pandas, Scikit-learn, and Power BI. Delivered solutions improving
P&L by 14.35% and reporting accuracy by 40%. M.Tech. in Computational Data Science from IISc
Bangalore.
Experience
Cloudcraftz Apr 2025 - Oct 2025
Data Scientist Kolkata
Strategy Performance Reconciliation and Validation:
Challenge -Live trading strategies were actively running in the market, and it was essential to validate
whether their performance aligned with back-tested expectations.
Approach -Leveraged the in-house backtesting engine to re-run strategies on minute-level market data,

ROSHAN CHOWRASIA
+91-8334913863
roshanchowrasia12@gmail.com
linkedin.com/in/roshan-chowrasia-625b3625b
Summary
Data Scientist with 1.5+ years of industrial experience specializing in machine learning, statistical
optimization, and time series forecasting. Proven expertise in predictive modeling, ETL automation, and data
pipeli

In [73]:
from pathlib import Path
print(Path("data").resolve())

C:\Users\Roshan\Desktop\RAG\RAG-Tutorials\notebook\data
